In [ ]:
import glob
import shutil
import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from os import listdir, makedirs
from os.path import isfile, join, exists
%matplotlib inline
from tqdm import tqdm

In [ ]:
train_wd = os.path.join(os.getcwd(),'test')
test_files = [f for f in listdir(train_wd) if isfile(join(train_wd, f))]

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
tuples = list(zip(df["landmark_id"], df["id"]))
lm_dict = defaultdict(list)
for tup in tuples:
    lm_dict[tup[0]].append(tup[1])

#lm_dict.keys()

In [ ]:
import cv2
img_size = (32,32)
for i in tqdm([*lm_dict]):
    filename = str(i) + ".jpg"
    if filename in downloaded_files:
        p=os.path.join(train_wd,filename)
        img=cv2.imread(p)
        img = cv2.resize(img, img_size)
        imwrite(filename,img)

In [ ]:
#transfer 10% of data in train to a new folder called val
val_wd=os.path.join(os.getcwd(),'val')
if not exists(val_wd):
    makedirs(val_wd)

In [ ]:
val_files = downloaded_files[:len(downloaded_files)//100]
train_files = downloaded_files[len(downloaded_files)//100:]

In [ ]:
for key in tqdm([*lm_dict]): 
    newpath = os.path.join(train_wd, str(key))
    valpath = os.path.join(val_wd, str(key))

    for index, img in enumerate(lm_dict[key]):
        if index >= 50:
            break
        
        filename = str(img) + ".jpg"
        if exists(os.path.join(train_wd, filename)) and filename in train_files:
            if not exists(newpath):
                makedirs(newpath)
#             try:
#                 img=Image.open(os.path.join(train_wd, filename))
#                 img=img.convert('RGB')
#             except Exception:
#                 print ("Bad one here:", os.path.join(train_wd, filename))
#                 os.remove(os.path.join(train_wd, filename))
#                 continue
            shutil.move(os.path.join(train_wd, filename), os.path.join(newpath, filename))
        elif exists(os.path.join(train_wd, filename)) and filename in val_files:
            if not exists(valpath):
                makedirs(valpath)
#             try:
#                 img=Image.open(os.path.join(train_wd, filename))
#                 img=img.convert('RGB')
#             except Exception:
#                 print ("Bad one here:", os.path.join(train_wd, filename))
#                 os.remove(os.path.join(train_wd, filename))
#                 continue
            shutil.move(os.path.join(train_wd, filename), os.path.join(valpath, filename))

In [ ]:
#[*lm_dict]

In [ ]:
#shutil.move("path/to/current/file.foo", "path/to/new/destination/for/file.foo")

In [ ]:
# for key in tqdm([*lm_dict]):
#     newpath = os.path.join(train_wd, str(key))
# #     print(newpath)
#     valpath = os.path.join(val_wd, str(key))
#     path=None
#     if not exists(os.path.join(newpath)):
#         path = valpath
#     else:
#         path = newpath

#     for img in lm_dict[key]:
#         filename = str(img) + ".jpg"
#         if exists(os.path.join(path, filename)) and filename in train_files:
#             try:
#                 img=Image.open(os.path.join(path, filename))
#                 img=img.convert('RGB')
#             except Exception:
#                 print ("Bad one here:", os.path.join(path, filename))
#                 os.remove(os.path.join(path, filename))
#                 continue
#         elif exists(os.path.join(path, filename)) and filename in val_files:
#             try:
#                 img=Image.open(os.path.join(path, filename))
#                 img=img.convert('RGB')
#             except Exception:
#                 print ("Bad one here:", os.path.join(path, filename))
#                 os.remove(os.path.join(path, filename))
#                 continue

In [ ]:
for filename in downloaded_files:
    filename = os.path.join(os.getcwd(),"train",filename)
    try:
        pil_im = Image.open(filename, 'r')
#         plt.imshow(np.asarray(pil_im))
#         plt.show()
    except Exception:
        print(filename, "removed")
        os.remove(filename)

In [ ]:
 #/tmp/imagenet/classify_image_graph_def.pb

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time

import numpy as np
import tensorflow as tf

def load_graph(model_file):
    graph = tf.Graph()
    graph_def = tf.GraphDef()

    with open(model_file, "rb") as f:
        graph_def.ParseFromString(f.read())
    with graph.as_default():
        tf.import_graph_def(graph_def)

    return graph

def read_tensor_from_image_file(file_name, input_height=299, input_width=299,input_mean=0, input_std=255):
    input_name = "file_reader"
    output_name = "normalized"
    file_reader = tf.read_file(file_name, input_name)
    if file_name.endswith(".png"):
        image_reader = tf.image.decode_png(file_reader, channels = 3,
                                       name='png_reader')
    elif file_name.endswith(".gif"):
        image_reader = tf.squeeze(tf.image.decode_gif(file_reader,
                                                  name='gif_reader'))
    elif file_name.endswith(".bmp"):
        image_reader = tf.image.decode_bmp(file_reader, name='bmp_reader')
    else:
        image_reader = tf.image.decode_jpeg(file_reader, channels = 3,
                                        name='jpeg_reader')
    float_caster = tf.cast(image_reader, tf.float32)
    dims_expander = tf.expand_dims(float_caster, 0);
    resized = tf.image.resize_bilinear(dims_expander, [input_height, input_width])
    normalized = tf.divide(tf.subtract(resized, [input_mean]), [input_std])
    sess = tf.Session()
    result = sess.run(normalized)

    return result

def load_labels(label_file):
    label = []
    proto_as_ascii_lines = tf.gfile.GFile(label_file).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label

file_name = "./test/2eec3e1e3e745237.jpg"
model_file = "./output.pb"
label_file = "./labels.txt"
input_height = 299
input_width = 299
input_mean = 128
input_std = 128
input_layer = "Mul"
output_layer = "final_result"

graph = load_graph(model_file)
submission = open("submissions_2.csv", "w")
submission.write("id,landmarks\n")

for file_name in tqdm(test_files[2000:4000]):
    t = read_tensor_from_image_file("./test/" + file_name,
                                  input_height=input_height,
                                  input_width=input_width,
                                  input_mean=input_mean,
                                  input_std=input_std)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name);
    output_operation = graph.get_operation_by_name(output_name);

    with tf.Session(graph=graph) as sess:
        results = sess.run(output_operation.outputs[0],
                          {input_operation.outputs[0]: t})
    results = np.squeeze(results)
    
    top_k = results.argsort()[-5:][::-1]
    labels = load_labels(label_file)
    
    submission.write(str(file_name[:-4]) + "," + str(labels[top_k[0]]) + " " + str(results[top_k[0]]) + "\n")
    
submission.close()

In [ ]:
#len(test_files)